Go ahead and install all the required packages using pip installer

Now lets import all the packages 

In [1]:
import pandas as pd
import numpy as np
import json

2. make sure you load the csv file using pandas

In [2]:
df = pd.read_csv("factors_of_women.csv")
df.head()

,S.NO,STATES,YEAR,RAPE,KIDNAP & ABDUCTION,DOWRY DEATHS,CRUETLY BY HUSBAND OR HIS RELATIVES,HUMAN TRAFFICKING,ASSAULT ON MODESTY,SEXUAL HARRASSMENT,OTHERS,Total Crimes Against Women
0,1,Andhra Pradesh,2011,1442,1612,599,13376,0,0,3658,497,28246
1,2,Arunachal Pradesh,2011,42,60,0,18,0,0,0,0,171
2,3,Assam,2011,1700,3192,121,5246,2,0,8,21,11503
3,4,Bihar,2011,934,3050,1413,2607,10,0,11,23,10231
4,5,Chhattisgarh,2011,1053,365,104,834,2,0,174,15,4219


In [3]:
indian_states = json.load(open('india_states.geojson', 'r'))

state_id_map = {}
for feature in indian_states['features']:
    feature['id'] = feature['properties']['state_code']
    state_id_map[feature['properties']['st_nm']] = feature['id']
    
df['id'] = df['STATES'].apply(lambda x : state_id_map[x])

In [4]:
#df[df["YEAR"] == 2019]

In [5]:
consolidated_state = df.groupby('STATES')[["RAPE","KIDNAP & ABDUCTION","DOWRY DEATHS","CRUETLY BY HUSBAND OR HIS RELATIVES","HUMAN TRAFFICKING","ASSAULT ON MODESTY","SEXUAL HARRASSMENT","OTHERS"]].sum().reset_index()
consolidated_state

,STATES,RAPE,KIDNAP & ABDUCTION,DOWRY DEATHS,CRUETLY BY HUSBAND OR HIS RELATIVES,HUMAN TRAFFICKING,ASSAULT ON MODESTY,SEXUAL HARRASSMENT,OTHERS
0,A & N Islands,223,96,5,69,1,344,96,10
1,Andhra Pradesh,12728,9786,3053,96269,344,49750,25925,2531
2,Arunachal Pradesh,741,832,3,582,4,918,81,7
3,Assam,19428,55094,1742,107680,581,36528,2599,339
4,Bihar,9743,65137,13568,35800,78,3285,622,399
5,Chandigarh,695,1312,37,1182,8,772,182,7
6,Chhattisgarh,15427,14621,840,8622,134,17400,2993,100
7,D&N Haveli,54,73,5,36,13,36,6,5
8,Daman & Diu,47,33,2,32,2,32,11,28
9,Delhi UT,15308,38271,1357,34395,117,29938,6878,110


In [6]:
consolidated_year = df[["STATES","YEAR", "RAPE","KIDNAP & ABDUCTION","DOWRY DEATHS","CRUETLY BY HUSBAND OR HIS RELATIVES","HUMAN TRAFFICKING","ASSAULT ON MODESTY","SEXUAL HARRASSMENT","OTHERS"]]

df_melted = pd.melt(consolidated_year, id_vars=['STATES','YEAR'], value_vars=["RAPE","KIDNAP & ABDUCTION","DOWRY DEATHS","CRUETLY BY HUSBAND OR HIS RELATIVES","HUMAN TRAFFICKING","ASSAULT ON MODESTY","SEXUAL HARRASSMENT","OTHERS"], var_name='CRIME_TYPE', value_name='VALUE')
#state_total = df.groupby('STATES')['VALUE'].sum().reset_index()
crime_type_total = df_melted.groupby('STATES')['VALUE'].sum().reset_index()

#df_melted[(df_melted["YEAR"] == 2011) and (df_melted["STATES"] == Andhra Pradesh)]

#df_combined[df_combined["STATES"] == "Bihar"]


In [7]:
df_combined = df_melted.merge(crime_type_total, on='STATES', how='left')
df_melted['PERCENTAGE'] = ((df_combined['VALUE_x'] / df_combined['VALUE_y'])*100).round(1)

#df_combined[df_combined["STATES"] == "Bihar"]

In [8]:
df_melted[df_melted["STATES"] == "Bihar"]


,STATES,YEAR,CRIME_TYPE,VALUE,PERCENTAGE
3,Bihar,2011,RAPE,934,0.7
39,Bihar,2012,RAPE,927,0.7
75,Bihar,2013,RAPE,1128,0.9
111,Bihar,2014,RAPE,1127,0.9
147,Bihar,2015,RAPE,1041,0.8
...,...,...,...,...,...
3005,Bihar,2017,OTHERS,24,0.0
3041,Bihar,2018,OTHERS,49,0.0
3077,Bihar,2019,OTHERS,40,0.0
3113,Bihar,2020,OTHERS,30,0.0


In [23]:
filter_df = df[df["YEAR"] == 2013] #filtered_data = filter_df

df_melted = pd.melt(df, id_vars=['STATES',"YEAR"], value_vars=["RAPE","KIDNAP & ABDUCTION","DOWRY DEATHS","CRUETLY BY HUSBAND OR HIS RELATIVES","HUMAN TRAFFICKING","ASSAULT ON MODESTY","SEXUAL HARRASSMENT","OTHERS"], var_name='CRIME_TYPE', value_name='VALUE')

#pie = px.pie(df, values='pop', names='country', title='Population of European continent')
#df_melted[df_melted["STATES"] == "Bihar"]
#max_crm = df_melted.groupby(["STATES"])['VALUE'].idxmax()
#df_melted.loc[max_crm].reset_index(drop=True)
filtered_data = df_melted[df_melted["STATES"].isin("Bihar")]
#df_melted[df_melted["STATES"] == "Bihar"]

,STATES,YEAR,CRIME_TYPE,VALUE
3,Bihar,2011,RAPE,934
39,Bihar,2012,RAPE,927
75,Bihar,2013,RAPE,1128
111,Bihar,2014,RAPE,1127
147,Bihar,2015,RAPE,1041
...,...,...,...,...
3005,Bihar,2017,OTHERS,24
3041,Bihar,2018,OTHERS,49
3077,Bihar,2019,OTHERS,40
3113,Bihar,2020,OTHERS,30


In [15]:
consolidated_states = df.groupby('YEAR')[["RAPE","KIDNAP & ABDUCTION","DOWRY DEATHS","CRUETLY BY HUSBAND OR HIS RELATIVES","HUMAN TRAFFICKING","ASSAULT ON MODESTY","SEXUAL HARRASSMENT","OTHERS"]].sum().reset_index()

melted = pd.melt(consolidated_states, id_vars=['YEAR'], value_vars=["RAPE","KIDNAP & ABDUCTION","DOWRY DEATHS","CRUETLY BY HUSBAND OR HIS RELATIVES","HUMAN TRAFFICKING","ASSAULT ON MODESTY","SEXUAL HARRASSMENT","OTHERS"], var_name='CRIME_TYPE', value_name='VALUE')

new_df = melted.groupby("CRIME_TYPE")["VALUE"].sum().reset_index()
#melted[melted["YEAR"] == 2012]
consolidated_states

,YEAR,RAPE,KIDNAP & ABDUCTION,DOWRY DEATHS,CRUETLY BY HUSBAND OR HIS RELATIVES,HUMAN TRAFFICKING,ASSAULT ON MODESTY,SEXUAL HARRASSMENT,OTHERS
0,2011,24206,35565,8618,99135,257,0,8570,2435
1,2012,24923,38262,8233,106527,59,45344,9173,2563
2,2013,33707,51881,8083,118866,31,70739,12589,2579
3,2014,36735,57311,10050,122877,13,82235,21938,2070
4,2015,34651,59277,7634,113403,6,82422,24041,2424
5,2016,38947,64519,7621,110378,659,84746,27344,2214
6,2017,32559,66333,7466,104551,662,86001,7451,1536
7,2018,33356,72751,7166,103272,854,89097,6992,1459
8,2019,32033,72780,7115,125298,966,88367,6939,1185
9,2020,28046,62300,6966,111549,646,85392,7065,868


In [11]:
filter_df = df[df["YEAR"] == 2012] #filtered_data = filter_df

pd.melt(filter_df, id_vars=["YEAR"], value_vars=["RAPE","KIDNAP & ABDUCTION","DOWRY DEATHS","CRUETLY BY HUSBAND OR HIS RELATIVES","HUMAN TRAFFICKING","ASSAULT ON MODESTY","SEXUAL HARRASSMENT","OTHERS"], var_name='CRIME_TYPE', value_name='VALUE')

,YEAR,CRIME_TYPE,VALUE
0,2012,RAPE,1341
1,2012,RAPE,46
2,2012,RAPE,1716
3,2012,RAPE,927
4,2012,RAPE,1034
...,...,...,...
283,2012,OTHERS,2
284,2012,OTHERS,3
285,2012,OTHERS,24
286,2012,OTHERS,0


In [12]:

df[["STATES", "YEAR", "Total Crimes Against Women"]]


df[df["STATES"]=="Andhra Pradesh"][["YEAR","Total Crimes Against Women"]]
    

,YEAR,Total Crimes Against Women
0,2011,28246
36,2012,28171
72,2013,32809
108,2014,16512
144,2015,15931
180,2016,16362
216,2017,17909
252,2018,16438
288,2019,17746
324,2020,17089


In [13]:
indian_states = json.load(open('india_states.geojson', 'r'))


state_id_map = {}
for feature in indian_states['features']:
    feature['id'] = feature['properties']['state_code']
    state_id_map[feature['properties']['st_nm']] = feature['id']
    
state_id_map
    

{'Telangana': 0,
 'A & N Islands': 35,
 'Andhra Pradesh': 28,
 'Arunachal Pradesh': 12,
 'Assam': 18,
 'Bihar': 10,
 'Chhattisgarh': 22,
 'Daman & Diu': 25,
 'Goa': 30,
 'Gujarat': 24,
 'Haryana': 6,
 'Himachal Pradesh': 2,
 'Jammu & Kashmir': 1,
 'Jharkhand': 20,
 'Karnataka': 29,
 'Kerala': 32,
 'Lakshadweep': 31,
 'Madhya Pradesh': 23,
 'Maharashtra': 27,
 'Manipur': 14,
 'Chandigarh': 4,
 'Puducherry': 34,
 'Punjab': 3,
 'Rajasthan': 8,
 'Sikkim': 11,
 'Tamil Nadu': 33,
 'Tripura': 16,
 'Uttar Pradesh': 9,
 'Uttarakhand': 5,
 'West Bengal': 19,
 'Odisha': 21,
 'D&N Haveli': 26,
 'Meghalaya': 17,
 'Mizoram': 15,
 'Nagaland': 13,
 'Delhi UT': 7}

In [14]:
state_id_map = json.load(open('state_id_map.json','r'))
state_id_map

FileNotFoundError: [Errno 2] No such file or directory: 'state_id_map.json'

In [ ]:
pre_df = df.groupby('YEAR')['Total Crimes Against Women'].sum().reset_index()
df[[]]

,STATES,Total Crimes Against Women
0,A & N Islands,1291
1,Andhra Pradesh,224965
2,Arunachal Pradesh,3431
3,Assam,241995
4,Bihar,161270
5,Chandigarh,4248
6,Chhattisgarh,66471
7,D&N Haveli,396
8,Daman & Diu,205
9,Delhi UT,136241


In [ ]:
future_years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026]    
new_yr = future_years.copy()
new_yr = new_yr[new_yr.index(2017) : new_yr.index(2021)+1]
np.array(new_yr).reshape(-1, 1).astype('int64')

array([[2017],
       [2018],
       [2019],
       [2020],
       [2021],
       [2022],
       [2023],
       [2024],
       [2025],
       [2026]], dtype=int64)

In [ ]:
x = pre_df['YEAR'].values.reshape(-1, 1)
y = pre_df["Total Crimes Against Women"]
x

array([[2011],
       [2012],
       [2013],
       [2014],
       [2015],
       [2016],
       [2017],
       [2018],
       [2019],
       [2020],
       [2021]], dtype=int64)

In [ ]:
pre_df["Total Crimes Against Women"]
pre_df[(pre_df['YEAR'] >= 2016) & (pre_df['YEAR'] <= 2020)]

,YEAR,Total Crimes Against Women
5,2016,338954
6,2017,359849
7,2018,378277
8,2019,405861
9,2020,371503
